## Function: `schedule_surgeries`

This function defines and solves an optimization model for scheduling surgeries over a specified horizon. It prioritizes surgeries based on patient waiting times and the urgency associated with each surgery type.

### Inputs:

- `surgeries_df` (pandas DataFrame): Contains each surgery's details, including a unique identifier (`surgery_id`), the type (`surgery_type`), and the admitted date represented as a negative integer for days waited.
- `surgery_type_attributes` (Dictionary): Stores attributes for each surgery type, such as expected duration (`duration`) and priority (`priority`).
- `current_day` (Integer): The current day number, serving as the starting point for the scheduling period.
- `original_capacities` (pandas DataFrame): The original available hours for each room on each day.
- `current_capacities` (pandas DataFrame): The current available hours for each room, adjusted as surgeries are scheduled.
- `days_ahead` (Integer): The number of days to include in the scheduling horizon.
- `beta` (double): Value between 0-1 which influences how much we prioritize based on priority values.

### Outputs:

- `scheduled_surgeries_df` (pandas DataFrame): Contains the scheduled surgeries with their assigned day and room.
- `current_capacities` (pandas DataFrame): Updated to reflect the remaining capacities after scheduling.
- `unscheduled_surgeries_df` (pandas DataFrame): Contains the surgeries not scheduled in the current iteration.

### Objective Function:

The objective is to minimize the total weighted waiting time for all surgeries, thereby prioritizing urgent cases and those who have been waiting longer.

$$
\text{Minimize} \sum_{s \in S} \sum_{d \in D} \sum_{r \in R} x_{s,d,r} \cdot \beta \cdot (\text{days\_waited}_s + (d - \text{current\_day})) \cdot \text{priority}_s
$$

where:
- $S$ is the set of surgeries.
- $D$ is the set of days in the scheduling horizon.
- $R$ is the set of rooms.
- $x_{s,d,r}$ is a binary decision variable indicating if surgery $s$ is scheduled in room $r$ on day $d$.
- $\text{days\_waited}_s$ is the days waited since admission, with an offset to ensure no zero values.
- $\text{priority}_s$ is the priority level of the surgery.

### Constraints:

**Single Scheduling Constraint:** Each surgery is scheduled at most once within the scheduling horizon.

$$
\sum_{d \in D} \sum_{r \in R} x_{s,d,r} \leq 1 \quad \forall s \in S
$$

**Capacity Constraints:** The total duration of surgeries in each room on each day must not exceed the available capacity, with adjustments for next-day and subsequent-days scheduling.

$$
\sum_{s \in S} x_{s,d,r} \cdot \text{duration}_s \leq \min(\text{allowed\_capacity}_{d,r}, \text{current\_capacity}_{d,r}) \quad \forall d \in D, \forall r \in R
$$

where:
- $\text{allowed\_capacity}_{d,r}$ is either 90% or 20% of the original capacity of room $r$ on day $d$, depending on whether $d$ is the next day or a subsequent days till max look ahead days.
- $\text{current\_capacity}_{d,r}$ is the remaining capacity for room $r$ on day $d$, updated as surgeries are scheduled.

### Execution:

- The model is optimized using the Gurobi optimizer, and the function returns the detailed schedule of surgeries, the updated capacities, and the list of unscheduled surgeries for potential future scheduling.


In [51]:
GLOBAL_NEXT_DAY_CAP = 0.9
GLOBAL_LOOKAHEAD_CAP = 0.2

In [52]:
from gurobipy import Model, GRB
import pandas as pd

def schedule_surgeries(surgeries_df, surgery_type_attributes, current_day, original_capacities, current_capacities, days_ahead=10, beta=0.0):
    if 'admitted_date' not in surgeries_df.columns:
        raise ValueError("The 'admitted_date' column is missing from the surgeries DataFrame.")

    # Calculate initial wait times based on admitted_date
    surgeries_df['initial_days_waited'] = (surgeries_df['admitted_date']) + 1

    # Initialize the model
    m = Model("SurgeryScheduling")

    # Determine the range of days to schedule
    days = range(current_day, current_day + days_ahead)
    rooms = original_capacities['room'].unique().tolist()

    # Add decision variables for scheduling surgeries
    surgery_scheduled = m.addVars(surgeries_df.index.tolist(), days, rooms, vtype=GRB.BINARY, name="schedule")

    # Set the objective function to minimize the total waiting time including scheduling delay
    m.setObjective(sum(surgery_scheduled[s, d, r] * (surgeries_df.loc[s, 'initial_days_waited']* (d - current_day)) * surgery_type_attributes[surgeries_df.loc[s, 'surgery_type']]['priority']
                       for s in surgeries_df.index for d in days for r in rooms), GRB.MINIMIZE)

    # Constraints to ensure each surgery is only scheduled once
    for s in surgeries_df.index:
        m.addConstr(sum(surgery_scheduled[s, d, r] for d in days for r in rooms) <= 1, name=f"OneTime_{s}")

    # Dynamic capacity constraints
    for d in days:
        for r in rooms:
            original_capacity = original_capacities.loc[(original_capacities['day'] == d) & (original_capacities['room'] == r), 'capacity'].values[0]
            
            # Calculate the percentage of the day already scheduled
            current_capacity = current_capacities.loc[(current_capacities['day'] == d) & (current_capacities['room'] == r), 'capacity'].values[0]
            percentage_already_scheduled = (original_capacity - current_capacity) / original_capacity
            
            # Calculate the maximum additional capacity that can be scheduled
            if d == current_day + 1:
                additional_capacity_allowed = GLOBAL_NEXT_DAY_CAP * original_capacity - (percentage_already_scheduled * original_capacity)
            else:
                additional_capacity_allowed = GLOBAL_LOOKAHEAD_CAP * original_capacity - (percentage_already_scheduled * original_capacity)
            
            # Take the minimum of the additional capacity allowed or the current remaining capacity
            allowed_capacity = min(additional_capacity_allowed, current_capacity)

            m.addConstr(sum(surgery_scheduled[s, d, r] * surgery_type_attributes[surgeries_df.loc[s, 'surgery_type']]['duration']
                            for s in surgeries_df.index) <= allowed_capacity,
                        name=f"Capacity_{d}_{r}")

    # Optimize the model
    m.optimize()

    # Update current capacities and prepare the schedule output
    scheduled_surgeries = []
    for s in surgeries_df.index:
        for d in days:
            for r in rooms:
                if surgery_scheduled[s, d, r].X > 0.5:
                    scheduled_surgeries.append({'surgery_id': s, 'day': d, 'room': r, 'duration': surgery_type_attributes[surgeries_df.loc[s, 'surgery_type']]['duration']})
                    # Update remaining capacity
                    current_capacities.loc[(current_capacities['day'] == d) & (current_capacities['room'] == r), 'capacity'] -= surgery_type_attributes[surgeries_df.loc[s, 'surgery_type']]['duration']

    scheduled_surgeries_df = pd.DataFrame(scheduled_surgeries)

    # Filter out scheduled surgeries from the original DataFrame
    unscheduled_surgeries_df = surgeries_df[~surgeries_df.index.isin(scheduled_surgeries_df['surgery_id'])]

    return scheduled_surgeries_df, current_capacities, unscheduled_surgeries_df


In [53]:
surgery_type_attributes = {
    "Cardiovascular": {"duration": 4, "variance": 1.0, "priority": 3},
    "Neurological": {"duration": 2, "variance": 1.2, "priority": 2.5},
    "Orthopedic": {"duration": 3, "variance": 0.5, "priority": 1.5},
    "Gastrointestinal": {"duration": 4, "variance": 0.7, "priority": 2.3},
    "Cosmetic": {"duration": 5, "variance": 0.3, "priority": 1},
}

surgery_portfolio_df = pd.read_csv("surgery_portfolio.csv")
initial_room_capacities = pd.read_csv('OR_caps.csv')

In [54]:
current_day = 1  # Example starting point
days_ahead = 10  # How far ahead to schedule

# Call the scheduler function with initial capacities since it's the first run
scheduled_surgeries_df, updated_capacities, unscheduled_surgeries_df = schedule_surgeries(
    surgeries_df=surgery_portfolio_df,
    surgery_type_attributes=surgery_type_attributes,
    current_day=current_day,
    original_capacities=initial_room_capacities,
    current_capacities=initial_room_capacities.copy(),  # Clone for the first run
    days_ahead=days_ahead
)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 63 rows, 990 columns and 1980 nonzeros
Model fingerprint: 0xb5160019
Variable types: 0 continuous, 990 integer (990 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective -81.0000000
Presolve removed 34 rows and 912 columns
Presolve time: 0.00s
Presolved: 29 rows, 78 columns, 156 nonzeros
Variable types: 0 continuous, 78 integer (78 binary)
Found heuristic solution: objective -128.0000000

Root relaxation: objective -1.465000e+02, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI


Cutting planes:
  Gomory: 2
  Cover: 3
  MIR: 4
  StrongCG: 1
  RLT: 3

Explored 1 nodes (58 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 3: -135 -128 -81 
No other solutions better than -135

Optimal solution found (tolerance 1.00e-04)
Best objective -1.350000000000e+02, best bound -1.350000000000e+02, gap 0.0000%


In [55]:
scheduled_surgeries_df

,surgery_id,day,room,duration
0,6,2,2,3
1,10,2,2,2
2,11,2,2,2
3,19,2,0,4
4,24,2,1,2
5,27,2,0,2
6,28,2,1,4


In [56]:
unscheduled_surgeries_df

,surgery_type,admitted_date,duration,priority,variance,initial_days_waited
0,Orthopedic,-2,3,1.5,0.5,-1
1,Gastrointestinal,-6,4,2.3,0.7,-5
2,Neurological,-1,2,2.5,1.2,0
3,Neurological,0,2,2.5,1.2,1
4,Orthopedic,-5,3,1.5,0.5,-4
5,Cardiovascular,-6,4,3.0,1.0,-5
7,Orthopedic,-1,3,1.5,0.5,0
8,Cosmetic,-9,5,1.0,0.3,-8
9,Cosmetic,-10,5,1.0,0.3,-9
12,Cosmetic,0,5,1.0,0.3,1


In [57]:
updated_capacities

,Unnamed: 0,day,room,capacity
0,0,0,0,8
1,1,0,1,8
2,2,0,2,8
3,3,1,0,8
4,4,1,1,8
...,...,...,...,...
85,85,28,1,8
86,86,28,2,8
87,87,29,0,8
88,88,29,1,8
